### [3454 扫描线](https://leetcode.cn/problems/separate-squares-ii/solutions/3878861/fen-ge-zheng-fang-xing-ii-by-leetcode-so-baas/?envType=daily-question&envId=2026-01-14)  
<font color=#FF3366>扫描线</font>


In [ ]:
from typing import List
import bisect

class SegmentTree:
    def __init__(self, xs: List[int]):
        self.xs = xs 
        self.n = len(xs) - 1
        self.count = [0] * (4 * self.n)
        self.covered = [0] * (4 * self.n)
    
    def update(self, qleft, qright, qval, left, right, pos):
        if self.xs[right+1] <= qleft or self.xs[left] >= qright:
            return
        if qleft <= self.xs[left] and self.xs[right+1] <= qright:
            self.count[pos] += qval
        else: 
            mid = (left + right) // 2
            self.update(qleft, qright, qval, left, mid, pos*2 + 1)
            self.update(qleft, qright, qval, mid+1, right, pos*2 + 2)

        if self.count[pos] > 0:
            self.covered[pos] = self.xs[right + 1] - self.xs[left]
        else:
            if left == right:
                self.covered[pos] = 0
            else:
                self.covered[pos] = self.covered[pos * 2 + 1] + self.covered[pos * 2 + 2]
    
    def query(self):
        return self.covered[0]

class Solution:
    def separateSquares(self, squares: List[List[int]]) -> float:
        events = []
        xs_set = set()
        for x, y, l in squares:
            events.append((y, 1, x, x + l)) 
            events.append((y + l, -1, x, x + l))
            xs_set.update([x, x + l])
        xs = sorted(xs_set)
        
        seg_tree = SegmentTree(xs)
        events.sort()
        
        psum = []
        widths = []
        total_area = 0.0
        prev_y = events[0][0]
        
        # 扫描：计算总面积和记录中间状态
        for y, delta, xl, xr in events:
            length = seg_tree.query()
            total_area += length * (y - prev_y)
            seg_tree.update(xl, xr, delta, 0, seg_tree.n - 1, 0)
            # 记录前缀和和宽度
            psum.append(total_area)
            widths.append(seg_tree.query())
            prev_y = y
        
        # 计算目标面积（向上取整的一半）
        target = (total_area + 1) // 2
        # 二分查找第一个大于等于target的位置
        i = bisect.bisect_left(psum, target) - 1
        # 获取对应的面积、宽度和高度
        area = psum[i]
        width = widths[i]
        height = events[i][0]

        return height + (total_area - area * 2) / (width * 2.0)